In [4]:
import tensorflow as tf
from tensorflow.keras.applications import Xception, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_deepfake_detector_resnet50(input_shape=(128, 128, 3), learning_rate=0.0001):

    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # for layer in base_model.layers[:-20]:
    #     layer.trainable = False
    # for layer in base_model.layers[-20:]:
    #     layer.trainable = True

    x = base_model.output

    x = GlobalAveragePooling2D(name='global_avg_pool')(x)

    x = Dense(256, activation='relu', name='fc1_relu')(x)

    x = Dropout(0.5, name='dropout_1')(x) 

    predictions = Dense(1, activation='sigmoid', name='output_sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

def build_deepfake_detector_xception(input_shape=(128, 128, 3), learning_rate=0.0001):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

    # for layer in base_model.layers[:-20]:
    #     layer.trainable = False
    # for layer in base_model.layers[-20:]:
    #     layer.trainable = True

    x = base_model.output

    x = GlobalAveragePooling2D(name='global_avg_pool')(x)

    x = Dense(256, activation='relu', name='fc1_relu')(x)

    x = Dropout(0.5, name='dropout_1')(x) 

    predictions = Dense(1, activation='sigmoid', name='output_sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

if __name__ == '__main__':
    DATA_DIR = "data_pre"
    IMG_HEIGHT = 128
    IMG_WIDTH = 128
    CHANNELS = 3
    BATCH_SIZE = 32
    EPOCHS = 10

    # Load datasets
    train_ds = tf.keras.utils.image_dataset_from_directory(
        f"{DATA_DIR}/train",
        labels="inferred",
        label_mode="binary",
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=42
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        f"{DATA_DIR}/val",
        labels="inferred",
        label_mode="binary",
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    choice = input("Choose model (1 for Xception, 2 for ResNet50): ")
    if choice =='1':
        # Preprocess images for Xception
        preprocess = tf.keras.applications.xception.preprocess_input
        train_ds = train_ds.map(lambda x, y: (preprocess(x), y))
        val_ds = val_ds.map(lambda x, y: (preprocess(x), y))
        input_shape = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)
        model = build_deepfake_detector_resnet50(input_shape=input_shape)
        model.summary()
        history = model.fit(
            train_ds,
            epochs=EPOCHS,
            validation_data=val_ds
            )
    elif choice == '2':
        # Preprocess images for ResNet50
        preprocess = tf.keras.applications.resnet50.preprocess_input
        train_ds = train_ds.map(lambda x, y: (preprocess(x), y))
        val_ds = val_ds.map(lambda x, y: (preprocess(x), y))

        # Build and train model
        input_shape = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)
        model = build_deepfake_detector_resnet50(input_shape=input_shape)
        model.summary()
        history = model.fit(
            train_ds,
            epochs=EPOCHS,
            validation_data=val_ds
            )   


Found 22896 files belonging to 2 classes.
Found 2514 files belonging to 2 classes.


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 134, 134,  │          0 │ input_layer_3[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 32, 32,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 32, 32,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 32, 32,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 32, 32,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 32, 32,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_3_c

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 24,059,393 (91.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

Epoch 1/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 118s 112ms/step - accuracy: 0.6333 - loss: 0.6668 - val_accuracy: 0.5270 - val_loss: 0.8546
Epoch 2/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 63s 88ms/step - accuracy: 0.8599 - loss: 0.3174 - val_accuracy: 0.5040 - val_loss: 2.0201
Epoch 3/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 61s 85ms/step - accuracy: 0.9334 - loss: 0.1677 - val_accuracy: 0.5016 - val_loss: 2.4958
Epoch 4/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 62s 86ms/step - accuracy: 0.9556 - loss: 0.1132 - val_accuracy: 0.5076 - val_loss: 3.6155
Epoch 5/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 59s 83ms/step - accuracy: 0.9673 - loss: 0.0910 - val_accuracy: 0.5167 - val_loss: 2.7150
Epoch 6/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 62s 87ms/step - accuracy: 0.9776 - loss: 0.0660 - val_accuracy: 0.5052 - val_loss: 3.1399
Epoch 7/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 64s 89ms/step - accuracy: 0.9796 - loss: 0.0611 - val_accuracy: 0.5183 - val_loss: 3.9148
Epoch 8/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 63s 88ms/step - accuracy: 0.9795 - loss: 0.0581 

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import Xception, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

def build_deepfake_detector_resnet50(input_shape=(128, 128, 3), learning_rate=0.0001):

    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    for layer in base_model.layers[:-20]:
        layer.trainable = False
    for layer in base_model.layers[-20:]:
        layer.trainable = True

    x = base_model.output

    x = GlobalAveragePooling2D(name='global_avg_pool')(x)

    x = Dense(256, activation='relu', name='fc1_relu')(x)

    x = Dropout(0.5, name='dropout_1')(x) 

    predictions = Dense(1, activation='sigmoid', name='output_sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

def build_deepfake_detector_xception(input_shape=(128, 128, 3), learning_rate=0.0001):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

    for layer in base_model.layers[:-20]:
        layer.trainable = False
    for layer in base_model.layers[-20:]:
        layer.trainable = True

    x = base_model.output

    x = GlobalAveragePooling2D(name='global_avg_pool')(x)

    x = Dense(256, activation='relu', name='fc1_relu')(x)

    x = Dropout(0.5, name='dropout_1')(x) 

    predictions = Dense(1, activation='sigmoid', name='output_sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

if __name__ == '__main__':
    DATA_DIR = "data_pre"
    IMG_HEIGHT = 128
    IMG_WIDTH = 128
    CHANNELS = 3
    BATCH_SIZE = 32
    EPOCHS = 10

    # Load datasets
    train_ds = tf.keras.utils.image_dataset_from_directory(
        f"{DATA_DIR}/train",
        labels="inferred",
        label_mode="binary",
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=42
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        f"{DATA_DIR}/val",
        labels="inferred",
        label_mode="binary",
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    choice = input("Choose model (1 for Xception, 2 for ResNet50): ")
    if choice =='1':
        # Preprocess images for Xception
        preprocess = tf.keras.applications.xception.preprocess_input
        train_ds = train_ds.map(lambda x, y: (preprocess(x), y))
        val_ds = val_ds.map(lambda x, y: (preprocess(x), y))
        input_shape = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)
        model = build_deepfake_detector_resnet50(input_shape=input_shape)
        model.summary()
        history = model.fit(
            train_ds,
            epochs=EPOCHS,
            validation_data=val_ds
            )
    elif choice == '2':
        # Preprocess images for ResNet50
        preprocess = tf.keras.applications.resnet50.preprocess_input
        train_ds = train_ds.map(lambda x, y: (preprocess(x), y))
        val_ds = val_ds.map(lambda x, y: (preprocess(x), y))

        # Build and train model
        input_shape = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)
        model = build_deepfake_detector_resnet50(input_shape=input_shape)
        model.summary()
        history = model.fit(
            train_ds,
            epochs=EPOCHS,
            validation_data=val_ds
            )   


Found 22896 files belonging to 2 classes.
Found 2514 files belonging to 2 classes.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 134, 134,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 32, 32,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 32, 32,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 32, 32,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 32, 32,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 32, 32,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_3_c

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 9,456,129 (36.07 MB)

 Non-trainable params: 14,656,384 (55.91 MB)

Epoch 1/10


2025-06-03 09:43:43.774120: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6848', 4 bytes spill stores, 4 bytes spill loads

2025-06-03 09:43:43.796195: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6848', 12 bytes spill stores, 16 bytes spill loads



716/716 ━━━━━━━━━━━━━━━━━━━━ 51s 53ms/step - accuracy: 0.5986 - loss: 0.6997 - val_accuracy: 0.5294 - val_loss: 0.9894
Epoch 2/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 25s 35ms/step - accuracy: 0.8012 - loss: 0.4182 - val_accuracy: 0.4375 - val_loss: 1.7832
Epoch 3/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 25s 34ms/step - accuracy: 0.8978 - loss: 0.2418 - val_accuracy: 0.4590 - val_loss: 2.4102
Epoch 4/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 25s 35ms/step - accuracy: 0.9424 - loss: 0.1478 - val_accuracy: 0.4483 - val_loss: 3.1202
Epoch 5/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 25s 35ms/step - accuracy: 0.9602 - loss: 945.0408 - val_accuracy: 0.5032 - val_loss: 1.1275
Epoch 6/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 26s 36ms/step - accuracy: 0.9689 - loss: 0.0857 - val_accuracy: 0.4996 - val_loss: 1.4361
Epoch 7/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 26s 37ms/step - accuracy: 0.9748 - loss: 0.0685 - val_accuracy: 0.5008 - val_loss: 1.2193
Epoch 8/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 27s 37ms/step - accuracy: 0.9829 - loss: 0.0562 - val_accur

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import Xception, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2

def build_deepfake_detector_resnet50(input_shape=(128, 128, 3), learning_rate=0.0001):

    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    for layer in base_model.layers[:-20]:
        layer.trainable = False
    for layer in base_model.layers[-20:]:
        layer.trainable = True

    x = base_model.output

    x = GlobalAveragePooling2D(name='global_avg_pool')(x)

    x = Dense(256, activation='relu', name='fc1_relu')(x)

    x = Dropout(0.5, name='dropout_1')(x) 

    predictions = Dense(1, activation='sigmoid', name='output_sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

def build_deepfake_detector_xception(input_shape=(128, 128, 3), learning_rate=0.0001):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

    # for layer in base_model.layers[:-20]:
    #     layer.trainable = False
    # for layer in base_model.layers[-20:]:
    #     layer.trainable = True

    x = base_model.output

    x = GlobalAveragePooling2D(name='global_avg_pool')(x)

    x = Dense(256, activation='relu', kernel_regularizer=l2(0.001), name='fc1_relu')(x)

    x = Dropout(0.5, name='dropout_1')(x) 

    predictions = Dense(1, activation='sigmoid', name='output_sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

if __name__ == '__main__':
    DATA_DIR = "data_pre"
    IMG_HEIGHT = 128
    IMG_WIDTH = 128
    CHANNELS = 3
    BATCH_SIZE = 32
    EPOCHS = 10

    # Load datasets
    train_ds = tf.keras.utils.image_dataset_from_directory(
        f"{DATA_DIR}/train",
        labels="inferred",
        label_mode="binary",
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=42
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        f"{DATA_DIR}/val",
        labels="inferred",
        label_mode="binary",
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    choice = input("Choose model (1 for Xception, 2 for ResNet50): ")
    if choice =='1':
        # Preprocess images for Xception
        # data_augmentation = tf.keras.Sequential([
        #     layers.RandomFlip("horizontal"),
        #     layers.RandomRotation(0.1),
        #     layers.RandomZoom(0.1),
        # ])
        # train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
        preprocess = tf.keras.applications.xception.preprocess_input
        train_ds = train_ds.map(lambda x, y: (preprocess(x), y))
        val_ds = val_ds.map(lambda x, y: (preprocess(x), y))
        input_shape = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)
        model = build_deepfake_detector_resnet50(input_shape=input_shape)
        model.summary()
        history = model.fit(
            train_ds,
            epochs=EPOCHS,
            validation_data=val_ds
            )
    elif choice == '2':
        # Preprocess images for ResNet50
        preprocess = tf.keras.applications.resnet50.preprocess_input
        train_ds = train_ds.map(lambda x, y: (preprocess(x), y))
        val_ds = val_ds.map(lambda x, y: (preprocess(x), y))

        # Build and train model
        input_shape = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)
        model = build_deepfake_detector_resnet50(input_shape=input_shape)
        # model.summary()
        history = model.fit(
            train_ds,
            epochs=EPOCHS,
            validation_data=val_ds
            )   


Found 22896 files belonging to 2 classes.
Found 2514 files belonging to 2 classes.


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 134, 134,  │          0 │ input_layer_5[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 32, 32,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 32, 32,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 32, 32,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 32, 32,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 32, 32,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_3_c

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 9,456,129 (36.07 MB)

 Non-trainable params: 14,656,384 (55.91 MB)

Epoch 1/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 44s 47ms/step - accuracy: 0.5372 - loss: 0.7033 - val_accuracy: 0.5255 - val_loss: 0.6908
Epoch 2/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 25s 34ms/step - accuracy: 0.5842 - loss: 0.6696 - val_accuracy: 0.5489 - val_loss: 0.7047
Epoch 3/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 23s 32ms/step - accuracy: 0.6027 - loss: 0.6589 - val_accuracy: 0.5533 - val_loss: 0.6970
Epoch 4/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 25s 34ms/step - accuracy: 0.6220 - loss: 0.6444 - val_accuracy: 0.5123 - val_loss: 0.7365
Epoch 5/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 23s 32ms/step - accuracy: 0.6294 - loss: 0.6319 - val_accuracy: 0.5151 - val_loss: 0.8454
Epoch 6/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 24s 33ms/step - accuracy: 0.6439 - loss: 0.6185 - val_accuracy: 0.5127 - val_loss: 0.8784
Epoch 7/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 25s 35ms/step - accuracy: 0.6563 - loss: 0.6037 - val_accuracy: 0.5207 - val_loss: 0.8348
Epoch 8/10
716/716 ━━━━━━━━━━━━━━━━━━━━ 25s 34ms/step - accuracy: 0.6683 - loss: 0.5860 - 